In [54]:
import os
import openai
from dotenv import load_dotenv

load_dotenv("../apiKey.env")
openai.api_key = os.getenv("API_KEY")

def generation():
  response = openai.ChatCompletion.create(
  # model="gpt-3.5-turbo",
  model="gpt-4-1106-preview",
  messages=[
    {
      "role": "system",
      # "content": "Formulate prompts that state a clear mathematical operation in words or symbols. format prompt || expression\n\nGenerate with a mix of concepts as below \n\nSimple Arithmetic:\n\"Add 7 to 15.\" || \"7 + 15\"\nCompound Operations:\n\"First multiply 4 by 3, then add 7.\" || \"(4 * 3) + 7\"\nUse of Brackets:\n\"Add 3 to 8, and then multiply the result by 2.\" || \"(3 + 8) * 2\"\nWord Problems:\n\"If you have 20 apples and give away 5, how many are left?\" || \"20 - 5\"\n Keep a consistant format. Don't number the results"
      "content": "Formulate prompts that state a clear mathematical operation in words or symbols. format prompt || expression\n\nGenerate with a mix of concepts as below \n\nSimple Arithmetic:\n\"Add 7 to 15.\" || \"7 + 15\"\nCompound Operations:\n\"First multiply 4 by 3, then add 7.\" || \"(4 * 3) + 7\"\nUse of Brackets:\n\"Add 3 to 8, and then multiply the result by 2.\" || \"(3 + 8) * 2\"\nSimple Word Problems:\n\"If you have 20 apples and give away 5, how many are left?\" || \"20 - 5\"\n Keep a consistant format. Don't number the results"
    },
    {
      "role": "user",
      "content": "Generate 5 calculator app prompts"
    },
  ],
  temperature=0.4,
  max_tokens=1066,
  top_p=1,
  frequency_penalty=1.22,
  presence_penalty=1.07
)
  return response

ModuleNotFoundError: No module named 'openai'

In [ ]:
iterations = 6

for n in range(iterations):
    res = generation()['choices'][0]['message']['content']
    with open('calcFineTune.txt', 'a') as file:
        file.write(res)



## FINE TUNE

In [ ]:
!pip install -r adapter_requirements.txt

In [1]:
import evaluate
import transformers
from transformers import BertTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM, BertForQuestionAnswering, AdapterConfig, AdapterType, T5ForConditionalGeneration, T5Tokenizer

from transformers.adapters import AdapterArguments, AdapterTrainer, AutoAdapterModel, setup_adapter_training
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from torch.utils.data import Dataset, DataLoader
import torch
import json


c:\Users\mekae\Desktop\CS\ML-DL\My Projects\Thesis Project II\adapter-finetune\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# tokenizer = BertTokenizer.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")
# model = BertForQuestionAnswering.from_pretrained("deepset/bert-large-uncased-whole-word-masking-squad2")

modelName = "google/flan-t5-large"
modelName = "google/flan-t5-base"
modelName = "google/flan-t5-small"

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


# tokenizer = AutoTokenizer.from_pretrained(modelName)
# model = AutoModelForSeq2SeqLM.from_pretrained(modelName)

# Add a new adapter for arithmetic expressions
config = AdapterConfig.load("houlsby", non_linearity="gelu", reduction_factor=16)
model.add_adapter("arithmetic_expressionV2", config=config)
model.train_adapter("arithmetic_expressionV2")
model.to('cuda')

NameError: name 'T5Tokenizer' is not defined

In [1]:
lines = []
data = []
with open("calcFineTune.txt", 'r') as file:
    lines = file.readlines()

for line in lines:
    # print(line)
    x = line.split("||")

    dataDict = {'prompt': x[0], 'expression': x[1]}
    data.append(dataDict)

data
# data = json.dumps(data,indent=4)
# with open("calcFineTuneDataJSON", 'w') as file:
    # json.dump(data, file, indent=4)
# data

[{'prompt': '"Subtract 9 from 25." ', 'expression': ' "25 - 9"\n'},
 {'prompt': '"First multiply 6 by 4, then divide the result by 2." ',
  'expression': ' "(6 * 4) / 2"\n'},
 {'prompt': '"Add 12 to the product of 5 and 3." ',
  'expression': ' "12 + (5 * 3)"\n'},
 {'prompt': '"If you have $50 and spend $20, how much money do you have left?" ',
  'expression': ' "$50 - $20"\n'},
 {'prompt': '"Divide the sum of 15 and 8 by the difference between 10 and 3." ',
  'expression': ' "(15 +8) / (10 -3)"\n'},
 {'prompt': '"Divide 36 by 6." ', 'expression': ' "36 / 6"\n'},
 {'prompt': '"If you have $50 and spend $20, how much money do you have left?" ',
  'expression': ' "$50 - $20"\n'},
 {'prompt': '"Multiply the number of days in a week (7) by the number of weeks in a month (4)." ',
  'expression': ' "7 * 4"\n'},
 {'prompt': '"Add up the ages of three people: John is 25 years old, Lisa is 32 years old, and Tom is 19 years old." ',
  'expression': ' "25 + 32 +19"\n'},
 {'prompt': '"Subtract 12 

In [ ]:
class ArithmeticDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = "Extract arithmetic expression: " + item['prompt'].strip()
        expression = item['expression'].strip()

        tokenized_input = self.tokenizer(prompt, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        tokenized_output = self.tokenizer(expression, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        return {
            "input_ids": tokenized_input.input_ids.squeeze(),
            "attention_mask": tokenized_input.attention_mask.squeeze(),
            "labels": tokenized_output.input_ids.squeeze()
        }


In [2]:
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels
    }

# Use this collate function in your DataLoader
dataset = ArithmeticDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_batch)



optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 1

NameError: name 'ArithmeticDataset' is not defined

In [ ]:
for batch in dataloader:
    print(batch['labels'])

tensor([[  96,  599, 1714,  ...,    0,    0,    0],
        [  96, 4448,    3,  ...,    0,    0,    0],
        [  96,  599, 2122,  ...,    0,    0,    0],
        ...,
        [  96,  599,  948,  ...,    0,    0,    0],
        [  96,  599, 2658,  ...,    0,    0,    0],
        [  96, 4165,    3,  ...,    0,    0,    0]])
tensor([[  96,  599, 2915,  ...,    0,    0,    0],
        [  96,  599,  536,  ...,    0,    0,    0],
        [  96, 1828, 1429,  ...,    0,    0,    0],
        ...,
        [  96, 1828, 1429,  ...,    0,    0,    0],
        [  96,  599, 2668,  ...,    0,    0,    0],
        [  96, 1828, 1429,  ...,    0,    0,    0]])
tensor([[  96, 2773, 1768,  ...,    0,    0,    0],
        [  96, 1808, 1429,  ...,    0,    0,    0],
        [  96, 4165,    3,  ...,    0,    0,    0],
        ...,
        [  96, 4165,    3,  ...,    0,    0,    0],
        [  96,  599, 4552,  ...,    0,    0,    0],
        [  96, 4448,    3,  ...,    0,    0,    0]])
tensor([[  96,  599,  

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch in dataloader:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}")


Epoch 1/1, Loss: 0.8868567798137664


In [ ]:
model.eval()
with torch.no_grad():  # Disable gradient tracking
    # Encode the prompt to token ids
    # token = tokenizer.encode("Extract arithmetic expression: \"can you do 3 plus 40 multiplied by 310\"", return_tensors='pt', max_length=max_length, truncation=True).to("cuda")
    token = tokenizer.encode("Extract arithmetic expression: \"Subtract 8 from 42\"", return_tensors='pt', max_length=50, truncation=True).to("cuda")
    print(token)
    # Generate the output sequence
    output_sequences = model.generate(input_ids=token, max_length=50)  # You can adjust max_length as needed
    print(output_sequences)
    # Decode the output to text
    output_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    print("Extracted Expression:", output_text)


tensor([[18742,     3,     9, 30922,    51,  7578,  3893,    10,    96, 25252,
          6471,    17,   505,    45,  6426,   121,     1]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], device='cuda:0')
Extracted Expression: 


Save Adapter

In [ ]:
model.save_adapter("./arithmetic_adapter", "arithmetic_expression")


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# path = "/content/drive/My Drive/My_Models/gpt1"
path = '/content/gdrive/My Drive/thesis-T5/'

# Or for PyTorch, save just the state_dict:
# torch.save(model.state_dict(), f'{path}/adapter_weights.pth')
model.save_adapter("./arithmetic_adapter", "arithmetic_expression")



In [16]:
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration
# from transformers import T5Tokenizer, T5ForConditionalGeneration, AdapterConfig
from torch.utils.data import Dataset, DataLoader

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Define the Dataset
class ArithmeticDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = "Extract arithmetic expression: " + item['prompt'].strip()
        expression = item['expression'].strip()

        tokenized_input = self.tokenizer(prompt, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        tokenized_output = self.tokenizer(expression, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        return {
            "input_ids": tokenized_input.input_ids.squeeze(),
            "attention_mask": tokenized_input.attention_mask.squeeze(),
            "labels": tokenized_output.input_ids.squeeze()
        }


# Your data loading code here
lines = []
data = []
with open("calcFineTune_2.txt", 'r') as file:
    lines = file.readlines()

for line in lines:
    print(line)
    x = line.split("||")

    dataDict = {'prompt': x[0], 'expression': x[1]}
    data.append(dataDict)


dataset = ArithmeticDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 15

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


"Subtract 9 from 25." || "25 - 9"

"First multiply 6 by 4, then divide the result by 2." || "(6 * 4) / 2"

"Add 12 to the product of 5 and 3." || "12 + (5 * 3)"

"If you have $50 and spend $20, how much money do you have left?" || "$50 - $20"

"Divide the sum of 15 and 8 by the difference between 10 and 3." || "(15 +8) / (10 -3)"

"Divide 36 by 6." || "36 / 6"

"If you have $50 and spend $20, how much money do you have left?" || "$50 - $20"

"Multiply the number of days in a week (7) by the number of weeks in a month (4)." || "7 * 4"

"Add up the ages of three people: John is 25 years old, Lisa is 32 years old, and Tom is 19 years old." || "25 + 32 +19"

"Subtract 12 from 28." || "28 - 12"

"Divide 36 by 4." || "36 / 4"

"Multiply the sum of 5 and 9 by 2." || "(5 +9) *2"

"If you have $50 and spend $25, how much money do you have left?" || "$50 - $25"

"Solve for x: (3x +10) = 22." || "x = (22-10)/3"1. "Subtract 9 from 42." || "42 - 9"

"Divide the sum of 10 and 5 by the difference bet

In [17]:
for epoch in range(num_epochs):
    total_loss = 0
    print(f"Starting Epoch {epoch+1}/{num_epochs}")

    for i, batch in enumerate(dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        predicted_ids = torch.argmax(outputs.logits, dim=-1)

        for i in range(predicted_ids.shape[0]):  # Loop over each item in the batch
            predicted_text = tokenizer.decode(predicted_ids[i], skip_special_tokens=True)
            print(f"Predicted text for input {i}: {predicted_text}")

        # print(outputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if i % 10 == 0:  # Adjust print frequency as needed
            print(f"Batch {i}, Loss: {loss.item()}")

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss}")

Starting Epoch 1/15
Predicted text for input 0: De is 8" Extract. " " " " " " " "
Predicted text for input 1: AddA to 9"" =.  "   " " " " " " " " " "
Predicted text for input 2: How dividedx 5  " " " " " " " " " " " " "   " " " " " " " " " " "
Predicted text for input 3: First100)x 1004) then- then and 3)"c
Predicted text for input 4: How9) times 4 plus 6? Extract. " " " " " " " " " "         " " "
Predicted text for input 5: If  $  =  " " " " " " " " "     "
Predicted text for input 6: TrueFiA)5) plus25 = Fal True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True True

KeyboardInterrupt: 

In [18]:
torch.save(model.state_dict(),"t5-base-weights_new_data-11-12-2023_1.pth")

In [26]:
model.eval()
with torch.no_grad():  # Disable gradient tracking
    # Encode the prompt to token ids
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # token = tokenizer.encode("Extract arithmetic expression: \"Subtract 8 from 42\"", return_tensors='pt').to(device)
    # token = tokenizer.encode("Extract arithmetic expression: \"Subtract 8 from 42\"", return_tensors='pt', padding='max_length', max_length=256, truncation=True).to(device)
    # token = tokenizer.encode("Extract arithmetic expression: \"what is 45233 plus 3332\"", return_tensors='pt', padding='max_length', max_length=256, truncation=True).to(device)
    # token = tokenizer.encode("Extract arithmetic expression: \"can you do 538 times 29 minus 23 then multiply by 12\"", return_tensors='pt', padding='max_length', max_length=256, truncation=True).to(device)
    token = tokenizer.encode("Extract arithmetic expression: \"what is 538 plus 29 minus 34\"", return_tensors='pt', padding='max_length', max_length=256, truncation=True).to(device)
    # tokenized_input = self.tokenizer(prompt, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

    # print(token)
    # Generate the output sequence
    output_sequences = model.generate(input_ids=token, max_length=256)  # You can adjust max_length as needed
    # print(output_sequences)
    # Decode the output to text
    output_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    print("Extracted Expression:", output_text)


Extracted Expression: "538 + 29 - 34"


In [26]:
# Example prompt
input_text = "Extract arithmetic expression: 'What is 3 plus 4 minus 3?'"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors='pt',max_length=512, truncation=True).to(device)

# Set the model to evaluation mode
model.eval()

# Generate output sequence
with torch.no_grad():  # Disable gradient tracking for inference
    print(input_ids)
    output_ids = model.generate(input_ids=input_ids, max_length=512)
    print(output_ids)
# Decode the output to text
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output_text)

print("Output:", output_text)


tensor([[18742,     3,     9, 30922,    51,  7578,  3893,    10,     3,    31,
          5680,    19,   220,   303,   314,     3, 14078,   220,    58,    31,
             1]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 